<a href="https://colab.research.google.com/github/VintageGold/Outages_Prediction/blob/master/Combing_Datasets_Generalizing_and_Aggregation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## In this notebook PJM's Outage and Usage data will be combined with NOAA's weather data.  


The generalizations are in the below sections.



1. Usage - Sum
2. Weather - Mean




# Import Libraries and setup functions

In [ ]:
import matplotlib.pyplot as plt 
import matplotlib.style as style
style.use('dark_background')

In [ ]:
import os
from os import listdir
import pandas as pd
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/My\ Drive/
os. getcwd()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive


'/content/drive/My Drive'

In [ ]:
import datetime as dt
def convert_dt(df,col_name=None,year=True,month=True):
    df[col_name] = pd.to_datetime(df[col_name])
    
    df['plot_date'] = pd.to_datetime(df[col_name].dt.strftime('%Y-%m-%d'))
    df['agg_date'] = df[col_name].dt.strftime('%m%d%Y')

    if year:
        df['year'] = df['agg_date'].str[-4:]
    if month:
        df['month'] = df['agg_date'].str[0:2]
        df = df.astype({'month': 'int64'})
    
    df = df.drop(columns='agg_date')
    return df

def get_month_season(df):
    df_month = pd.read_csv('UMBC_Energy/Universal_Data/months.csv')
    df = df.merge(df_month,on='month',how='left')
    df = df.astype({'month': 'object'})
    return df

In [ ]:
def get_mkt_region(mkt):
    
    if mkt == 'MIDATL':
        return  'Mid Atlantic - Dominion'
    
    elif mkt == 'SOUTH':
        return 'Mid Atlantic - Dominion'
    
    elif mkt == 'WEST':
        return 'Western'
    
    elif mkt == 'RTO':
        return 'PJM RTO'

## Load PJM Outage Data

In [ ]:
df_outage = pd.read_csv('UMBC_Energy/forecast_matched_execution.csv',index_col=[0])

df_outage = convert_dt(df_outage,col_name='forecast_execution_date_ept')
df_outage = get_month_season(df_outage)
df_outage

,forecast_execution_date_ept,forecast_date,region,total_outages_mw,planned_outages_mw,maintenance_outages_mw,forced_outages_mw,execution_date,forecast_just_date,date_match,plot_date,year,month,month_name,season
0,2015-12-31,2015-12-31,Mid Atlantic - Dominion,5063,1525,2827,711,12312015,12312015,yes,2015-12-31,2015,12,Dec,Winter
1,2015-12-31,2015-12-31,PJM RTO,15599,3696,6137,5766,12312015,12312015,yes,2015-12-31,2015,12,Dec,Winter
2,2015-12-31,2015-12-31,Western,10536,2171,3310,5055,12312015,12312015,yes,2015-12-31,2015,12,Dec,Winter
3,2015-12-30,2015-12-30,Mid Atlantic - Dominion,7178,1525,3873,1780,12302015,12302015,yes,2015-12-30,2015,12,Dec,Winter
4,2015-12-30,2015-12-30,PJM RTO,17116,3721,7481,5914,12302015,12302015,yes,2015-12-30,2015,12,Dec,Winter
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5572,2020-01-02,2020-01-02,Western,9459,495,6627,2337,1022020,1022020,yes,2020-01-02,2020,1,Jan,Winter
5573,2020-01-02,2020-01-02,PJM RTO,14364,964,8527,4873,1022020,1022020,yes,2020-01-02,2020,1,Jan,Winter
5574,2020-01-01,2020-01-01,Western,6735,495,4040,2200,1012020,1012020,yes,2020-01-01,2020,1,Jan,Winter
5575,2020-01-01,2020-01-01,Mid Atlantic - Dominion,6756,469,4424,1863,1012020,1012020,yes,2020-01-01,2020,1,Jan,Winter


##Load Usage Data

In [ ]:
df_usage_2015 = pd.read_excel('UMBC_Energy/Energy_usage/Usage_2015_2020.xlsx', 'hrl_load_metered_2015')
df_usage_2016 = pd.read_excel('UMBC_Energy/Energy_usage/Usage_2015_2020.xlsx', 'hrl_load_metered_2016')
df_usage_2017 = pd.read_excel('UMBC_Energy/Energy_usage/Usage_2015_2020.xlsx', 'hrl_load_metered_2017')
df_usage_2018 = pd.read_excel('UMBC_Energy/Energy_usage/Usage_2015_2020.xlsx', 'hrl_load_metered_2018')
df_usage_2019 = pd.read_excel('UMBC_Energy/Energy_usage/Usage_2015_2020.xlsx', 'hrl_load_metered_2019')
df_usage_2020 = pd.read_excel('UMBC_Energy/Energy_usage/Usage_2015_2020.xlsx', 'hrl_load_metered_2020')
df_usage = pd.concat([df_usage_2015,df_usage_2016,df_usage_2017,df_usage_2018,df_usage_2019,df_usage_2020])
df_usage = convert_dt(df_usage, col_name='datetime_beginning_ept')
df_usage = get_month_season(df_usage)
df_usage = df_usage.rename({'mw':'usage_mw'})
df_usage = df_usage.set_index('datetime_beginning_ept')

In [ ]:
df_usage

,datetime_beginning_utc,nerc_region,mkt_region,zone,load_area,mw,is_verified,plot_date,year,month,month_name,season
datetime_beginning_ept,,,,,,,,,,,,
2015-01-01 00:00:00,2015-01-01 05:00:00,RFC,MIDATL,DPL,DPL,2437.433,True,2015-01-01,2015,1,Jan,Winter
2015-01-01 00:00:00,2015-01-01 05:00:00,RFC,MIDATL,PEP,PEP,3561.818,True,2015-01-01,2015,1,Jan,Winter
2015-01-01 00:00:00,2015-01-01 05:00:00,RFC,WEST,AEP,AEP,16176.032,True,2015-01-01,2015,1,Jan,Winter
2015-01-01 00:00:00,2015-01-01 05:00:00,RFC,MIDATL,AE,AE,1180.082,True,2015-01-01,2015,1,Jan,Winter
2015-01-01 00:00:00,2015-01-01 05:00:00,RFC,MIDATL,BC,BC,3940.727,True,2015-01-01,2015,1,Jan,Winter
...,...,...,...,...,...,...,...,...,...,...,...,...
2020-07-01 23:00:00,2020-07-02 03:00:00,RFC,MIDATL,RECO,RECO,182.180,False,2020-07-01,2020,7,Jul,Summer
2020-07-01 23:00:00,2020-07-02 03:00:00,RFC,MIDATL,PEP,SMECO,379.719,False,2020-07-01,2020,7,Jul,Summer
2020-07-01 23:00:00,2020-07-02 03:00:00,RFC,MIDATL,PL,UGI,116.904,False,2020-07-01,2020,7,Jul,Summer


In [ ]:
df_usage.to_csv('UMBC_Energy/Energy_usage/Usage_months_seasons_ept.csv',index=True,sep=',')

In [ ]:
#Translate mkt_region from usage dataset to region in terms of outage dataset
df_usage['region'] = df_usage['mkt_region'].apply(lambda x: get_mkt_region(x))

In [ ]:
#Check Math from aggregation below
df_usage.query('region == "Mid Atlantic - Dominion" & plot_date == "2015-01-01"').sum()

nerc_region    RFCRFCRFCRFCRFCRFCRFCRFCRFCRFCRFCRFCSERCRFCRFC...
mkt_region     MIDATLMIDATLMIDATLMIDATLMIDATLMIDATLMIDATLMIDA...
zone           DPLPEPAEBCJCMEPEPLPLPNPSRECODOMDPLPEPAEBCJCMEP...
load_area      DPLPEPAEBCJCMEPEPLCOUGIPNPSRECODOMDPLPEPAEBCJC...
mw                                                   1.04974e+06
is_verified                                                  312
year           2015201520152015201520152015201520152015201520...
month                                                        312
month_name     JanJanJanJanJanJanJanJanJanJanJanJanJanJanJanJ...
season         WinterWinterWinterWinterWinterWinterWinterWint...
region         Mid Atlantic - DominionMid Atlantic - Dominion...
dtype: object

In [ ]:
group_df_usage_merge = df_usage.groupby(['region','plot_date']).sum()
 

In [ ]:
group_df_usage_merge

mw  is_verified
region                  plot_date                           
Mid Atlantic - Dominion 2015-01-01  1049743.647        312.0
                        2015-01-02  1042563.843        312.0
                        2015-01-03  1029206.526        312.0
                        2015-01-04   911121.292        312.0
                        2015-01-05  1070915.807        312.0
...                                         ...          ...
Western                 2020-06-27  1128450.113        216.0
                        2020-06-28  1128068.148        216.0
                        2020-06-29  1281585.133        168.0
                        2020-06-30  1296044.971        192.0
                        2020-07-01  1299621.398        120.0

[6027 rows x 2 columns]

In [ ]:
group_df_outage_merge = df_outage.groupby(['region','plot_date']).mean()


In [ ]:
group_df_outage_merge

total_outages_mw  ...  forecast_just_date
region                  plot_date                     ...                    
Mid Atlantic - Dominion 2015-05-26             14627  ...             5262015
                        2015-05-27             15001  ...             5272015
                        2015-06-03             12035  ...             6032015
                        2015-06-04             14090  ...             6042015
                        2015-06-05             13062  ...             6052015
...                                              ...  ...                 ...
Western                 2020-07-01              5305  ...             7012020
                        2020-07-02              5898  ...             7022020
                        2020-07-03              5535  ...             7032020
                        2020-07-04              5215  ...             7042020
                        2020-07-05              6463  ...             7052020

[5577 rows x 6 columns]

### Merge PJM Outage and Usage data together

In [ ]:
df_outage_usage = group_df_outage_merge.merge(group_df_usage_merge,left_index=True,right_index=True)
df_outage_usage = df_outage_usage.drop(columns='is_verified')
df_outage_usage = df_outage_usage.rename({'mw':'Usage_mw'})
df_outage_usage

total_outages_mw  ...           mw
region                  plot_date                     ...             
Mid Atlantic - Dominion 2015-05-26             14627  ...  1105822.407
                        2015-05-27             15001  ...  1158822.118
                        2015-06-03             12035  ...   876844.075
                        2015-06-04             14090  ...   876853.950
                        2015-06-05             13062  ...   888819.936
...                                              ...  ...          ...
Western                 2020-06-27             10283  ...  1128450.113
                        2020-06-28              8480  ...  1128068.148
                        2020-06-29              4768  ...  1281585.133
                        2020-06-30              5698  ...  1296044.971
                        2020-07-01              5305  ...  1299621.398

[5565 rows x 7 columns]

In [ ]:
df_outage_usage.reset_index(level=['plot_date','region'], inplace=True)
df_outage_usage_plot = df_outage_usage.drop(columns=['execution_date','forecast_just_date'])

In [ ]:
df_outage_usage['plot_date']

0      2015-05-26
1      2015-05-27
2      2015-06-03
3      2015-06-04
4      2015-06-05
          ...    
5560   2020-06-27
5561   2020-06-28
5562   2020-06-29
5563   2020-06-30
5564   2020-07-01
Name: plot_date, Length: 5565, dtype: datetime64[ns]

In [ ]:
df_outage_usage_plot = convert_dt(df_outage_usage_plot,col_name='plot_date')
df_outage_usage_plot = get_month_season(df_outage_usage_plot)
df_outage_usage_plot

,region,plot_date,total_outages_mw,planned_outages_mw,maintenance_outages_mw,forced_outages_mw,mw,year,month,month_name,season
0,Mid Atlantic - Dominion,2015-05-26,14627,9827,3328,1472,1105822.407,2015,5,May,Spring
1,Mid Atlantic - Dominion,2015-05-27,15001,9827,3336,1838,1158822.118,2015,5,May,Spring
2,Mid Atlantic - Dominion,2015-06-03,12035,5635,5506,894,876844.075,2015,6,Jun,Summer
3,Mid Atlantic - Dominion,2015-06-04,14090,5508,7496,1086,876853.950,2015,6,Jun,Summer
4,Mid Atlantic - Dominion,2015-06-05,13062,5508,6562,992,888819.936,2015,6,Jun,Summer
...,...,...,...,...,...,...,...,...,...,...,...
5560,Western,2020-06-27,10283,0,5787,4496,1128450.113,2020,6,Jun,Summer
5561,Western,2020-06-28,8480,0,4728,3752,1128068.148,2020,6,Jun,Summer
5562,Western,2020-06-29,4768,0,2809,1959,1281585.133,2020,6,Jun,Summer
5563,Western,2020-06-30,5698,0,3533,2165,1296044.971,2020,6,Jun,Summer


In [ ]:
df_outage_usage_plot.to_csv('UMBC_Energy/Model_ready/Outage_Usage_ept.csv',index=False)

In [ ]:
df_outage_usage_plot = df_outage_usage_plot.set_index('plot_date')

##Load NOAA Weather Data

In [ ]:
df_NOAA = pd.read_csv('UMBC_Energy/Raw/Raw_All_NOAA.csv',sep='|')
df_NOAA = df_NOAA[df_NOAA.datacoverage >= .99].sort_index(ascending=True)
df_NOAA = convert_dt(df_NOAA,'date')
df_NOAA = get_month_season(df_NOAA)
df_NOAA['TDIFF'] = df_NOAA['TMAX'] - df_NOAA['TMIN']
df_NOAA['Code_station'] = df_NOAA['station'] + df_NOAA['Code']
df_NOAA


,station_dt_key,date,station,PRCP,SNOW,SNWD,AWND,TMAX,TMIN,elevation,mindate,maxdate,latitude,name,datacoverage,id,elevationUnit,longitude,state,location_type,location,State,Code,plot_date,year,month,month_name,season,TDIFF,Code_station
0,GHCND:US1DEKN0001_2015-01-01,2015-01-01,GHCND:US1DEKN0001,0.0,0.0,NaN,NaN,NaN,NaN,9.8,2009-09-01,2020-06-22,39.03257,"FELTON 3.6 NE, DE US",0.9991,GHCND:US1DEKN0001,METERS,-75.516933,Delaware,GHCND,US1DEKN0001,Delaware,DE,2015-01-01,2015,1,Jan,Winter,NaN,GHCND:US1DEKN0001DE
1,GHCND:USC00283662_2015-01-01,2015-01-01,GHCND:USC00283662,0.0,0.0,0.0,NaN,32.0,19.0,38.1,1893-01-01,2020-07-19,39.64415,"HAMMONTON 1 NE, NJ US",1.0000,GHCND:USC00283662,METERS,-74.804780,New Jersey,GHCND,USC00283662,New Jersey,NJ,2015-01-01,2015,1,Jan,Winter,13.0,GHCND:USC00283662NJ
2,GHCND:USC00442635_2015-01-01,2015-01-01,GHCND:USC00442635,0.0,NaN,NaN,NaN,41.0,27.0,3.0,1909-11-01,2015-10-01,37.38760,"EASTVILLE, VA US",1.0000,GHCND:USC00442635,METERS,-75.974400,Virginia,GHCND,USC00442635,Virginia,VA,2015-01-01,2015,1,Jan,Winter,14.0,GHCND:USC00442635VA
3,GHCND:USC00440327_2015-01-01,2015-01-01,GHCND:USC00440327,0.0,0.0,0.0,NaN,46.0,22.0,67.1,1893-01-01,2020-07-18,37.75000,"ASHLAND, VA US",1.0000,GHCND:USC00440327,METERS,-77.483300,Virginia,GHCND,USC00440327,Virginia,VA,2015-01-01,2015,1,Jan,Winter,24.0,GHCND:USC00440327VA
4,GHCND:USC00366233_2015-01-01,2015-01-01,GHCND:USC00366233,0.0,0.0,0.0,NaN,30.0,17.0,251.5,1892-05-01,2020-01-31,41.01720,"NEW CASTLE 1 N, PA US",1.0000,GHCND:USC00366233,METERS,-80.361500,Pennsylvania,GHCND,USC00366233,Pennsylvania,PA,2015-01-01,2015,1,Jan,Winter,13.0,GHCND:USC00366233PA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210227,GHCND:USC00461220_2020-07-06,2020-07-06,GHCND:USC00461220,0.0,0.0,0.0,NaN,95.0,64.0,443.5,1888-01-01,2020-07-19,38.98030,"BUCKHANNON, WV US",1.0000,GHCND:USC00461220,METERS,-80.220100,West Virginia,GHCND,USC00461220,West Virginia,WV,2020-07-06,2020,7,Jul,Summer,31.0,GHCND:USC00461220WV
210228,GHCND:USC00338822_2020-07-06,2020-07-06,GHCND:USC00338822,0.0,0.0,0.0,NaN,93.0,65.0,228.6,1893-02-01,2020-07-19,41.51830,"WAUSEON WATER PLANT, OH US",1.0000,GHCND:USC00338822,METERS,-84.145200,Ohio,GHCND,USC00338822,Ohio,OH,2020-07-06,2020,7,Jul,Summer,28.0,GHCND:USC00338822OH
210229,GHCND:USC00361920_2020-07-06,2020-07-06,GHCND:USC00361920,0.0,0.0,0.0,NaN,95.0,67.0,131.1,1840-02-01,2020-07-19,40.22580,"CARLISLE WATER PLANT, PA US",1.0000,GHCND:USC00361920,METERS,-77.189400,Pennsylvania,GHCND,USC00361920,Pennsylvania,PA,2020-07-06,2020,7,Jul,Summer,28.0,GHCND:USC00361920PA
210230,GHCND:USC00361485_2020-07-06,2020-07-06,GHCND:USC00361485,NaN,NaN,0.0,NaN,90.0,57.0,317.0,1884-11-01,2020-07-16,41.19220,"CLARION 3 SW, PA US",1.0000,GHCND:USC00361485,METERS,-79.436100,Pennsylvania,GHCND,USC00361485,Pennsylvania,PA,2020-07-06,2020,7,Jul,Summer,33.0,GHCND:USC00361485PA


## Merge NOAA dataset with selected weather station locations.  This notebook can be found 

In [ ]:
df_NOAA_Station_Trim = pd.read_csv('UMBC_Energy/Raw/Raw_Station_Location.csv',sep=',')
df_NOAA_Station_Trim = df_NOAA_Station_Trim.merge(df_NOAA,left_on = ['Key','Code','station'],right_on=['Code_station','Code','station'],how='left')
c = df_NOAA_Station_Trim.sort_values('plot_date',ascending=True)

In [ ]:
df_Code_mkt_region = pd.read_csv('UMBC_Energy/Raw/Code_mkt_regions.csv',sep=',')
df_NOAA_Station_Trim_mkt_region = df_NOAA_Station_Trim.merge(df_Code_mkt_region,on='Code',how='left')
df_NOAA_Station_Trim_mkt_region['region'] = df_NOAA_Station_Trim_mkt_region['mkt_region'].apply(lambda x: get_mkt_region(x))

df_NOAA_Station_Trim_mkt_region.to_csv('UMBC_Energy/Raw/df_NOAA_Station_Trim_mkt_region.csv',index=True)

df_NOAA_Station_Trim_mkt_region = df_NOAA_Station_Trim_mkt_region.groupby(['region','plot_date']).mean()
df_NOAA_Station_Trim_mkt_region
#df_NOAA_Station_Trim_mkt_region.groupby('Code').count()

PRCP      SNOW  ...  longitude      TDIFF
region                  plot_date                       ...                      
Mid Atlantic - Dominion 2015-01-01  0.000000  0.000000  ... -77.597622  14.377778
                        2015-01-02  0.000000  0.000000  ... -77.597622  17.955556
                        2015-01-03  0.073750  0.016216  ... -77.650879  15.651163
                        2015-01-04  0.474250  0.059459  ... -77.650879  15.674419
                        2015-01-05  0.075500  0.036842  ... -77.597622  26.355556
...                                      ...       ...  ...        ...        ...
Western                 2020-07-02  0.000909  0.000000  ... -81.898968  24.100000
                        2020-07-03  0.018000  0.000000  ... -82.066855  24.250000
                        2020-07-04  0.000000  0.000000  ... -81.866995  25.200000
                        2020-07-05  0.109091  0.000000  ... -81.898968  27.000000
                        2020-07-06  0.002222  0.000000  ... -81.929018  26.500000

[4028 rows x 11 columns]

In [ ]:
df_outage_usage 

,region,plot_date,total_outages_mw,planned_outages_mw,maintenance_outages_mw,forced_outages_mw,execution_date,forecast_just_date,mw
0,Mid Atlantic - Dominion,2015-05-26,14627,9827,3328,1472,5262015,5262015,1105822.407
1,Mid Atlantic - Dominion,2015-05-27,15001,9827,3336,1838,5272015,5272015,1158822.118
2,Mid Atlantic - Dominion,2015-06-03,12035,5635,5506,894,6032015,6032015,876844.075
3,Mid Atlantic - Dominion,2015-06-04,14090,5508,7496,1086,6042015,6042015,876853.950
4,Mid Atlantic - Dominion,2015-06-05,13062,5508,6562,992,6052015,6052015,888819.936
...,...,...,...,...,...,...,...,...,...
5560,Western,2020-06-27,10283,0,5787,4496,6272020,6272020,1128450.113
5561,Western,2020-06-28,8480,0,4728,3752,6282020,6282020,1128068.148
5562,Western,2020-06-29,4768,0,2809,1959,6292020,6292020,1281585.133
5563,Western,2020-06-30,5698,0,3533,2165,6302020,6302020,1296044.971


##Merge Weather and Outage Usage Datasets

In [ ]:
df_outage_usage = df_outage_usage.set_index(['region','plot_date'])

In [ ]:
#Lost rows because start date of weather is 
df_outage_usage_weather = df_outage_usage.merge(df_NOAA_Station_Trim_mkt_region,left_index=True,right_index=True)
df_outage_usage_weather = df_outage_usage_weather.drop(columns=['longitude','latitude','elevation','datacoverage'])
df_outage_usage_weather

total_outages_mw  ...      TDIFF
region                  plot_date                     ...           
Mid Atlantic - Dominion 2015-05-26             14627  ...  25.355556
                        2015-05-27             15001  ...  20.212766
                        2015-06-03             12035  ...  11.765957
                        2015-06-04             14090  ...  11.088889
                        2015-06-05             13062  ...  11.130435
...                                              ...  ...        ...
Western                 2020-06-27             10283  ...  22.636364
                        2020-06-28              8480  ...  15.363636
                        2020-06-29              4768  ...  16.000000
                        2020-06-30              5698  ...  22.363636
                        2020-07-01              5305  ...  22.400000

[3710 rows x 14 columns]

In [ ]:
df_outage_usage_weather = df_outage_usage_weather.reset_index()
df_outage_usage_weather = convert_dt(df_outage_usage_weather,col_name='plot_date')
df_outage_usage_weather = get_month_season(df_outage_usage_weather)
df_outage_usage_weather = df_outage_usage_weather.set_index('plot_date')
df_outage_usage_weather = df_outage_usage_weather.drop(columns=['execution_date','forecast_just_date'])
df_outage_usage_weather = df_outage_usage_weather.rename(columns={'mw':'Usage_mw'})

In [ ]:
df_outage_usage_weather

,region,total_outages_mw,planned_outages_mw,maintenance_outages_mw,forced_outages_mw,Usage_mw,PRCP,SNOW,SNWD,AWND,TMAX,TMIN,TDIFF,year,month,month_name,season
plot_date,,,,,,,,,,,,,,,,,
2015-05-26,Mid Atlantic - Dominion,14627,9827,3328,1472,1105822.407,0.004255,0.0,0.0,11.083333,83.595745,58.266667,25.355556,2015,5,May,Spring
2015-05-27,Mid Atlantic - Dominion,15001,9827,3336,1838,1158822.118,0.058723,0.0,0.0,10.833333,83.787234,63.574468,20.212766,2015,5,May,Spring
2015-06-03,Mid Atlantic - Dominion,12035,5635,5506,894,876844.075,0.222391,0.0,0.0,10.766667,64.957447,53.191489,11.765957,2015,6,Jun,Summer
2015-06-04,Mid Atlantic - Dominion,14090,5508,7496,1086,876853.950,0.112609,0.0,0.0,9.116667,65.511111,54.422222,11.088889,2015,6,Jun,Summer
2015-06-05,Mid Atlantic - Dominion,13062,5508,6562,992,888819.936,0.092889,0.0,0.0,6.366667,67.282609,56.152174,11.130435,2015,6,Jun,Summer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-06-27,Western,10283,0,5787,4496,1128450.113,0.254167,0.0,0.0,6.300000,82.909091,60.272727,22.636364,2020,6,Jun,Summer
2020-06-28,Western,8480,0,4728,3752,1128068.148,0.171667,0.0,0.0,3.600000,81.727273,66.363636,15.363636,2020,6,Jun,Summer
2020-06-29,Western,4768,0,2809,1959,1281585.133,0.073333,0.0,0.0,2.900000,81.090909,65.090909,16.000000,2020,6,Jun,Summer


In [ ]:
df_outage_usage_weather.to_csv('UMBC_Energy/Model_ready/Outage_Usage_Weather_ept.csv',index=True)

#Validation for Model Notebooks.
a = df_outage_usage_weather['2016-01-01':'2020-01-02']
a[a.region == 'Mid Atlantic - Dominion']

,region,total_outages_mw,planned_outages_mw,maintenance_outages_mw,forced_outages_mw,Usage_mw,PRCP,SNOW,SNWD,AWND,TMAX,TMIN,TDIFF,year,month,month_name,season
plot_date,,,,,,,,,,,,,,,,,
2016-01-01,Mid Atlantic - Dominion,4480,1525,1796,1159,899146.903,0.002222,0.010526,0.000000,7.875000,47.644444,34.200000,13.444444,2016,1,Jan,Winter
2016-01-02,Mid Atlantic - Dominion,6389,1537,4010,842,942882.797,0.000000,0.000000,0.000000,6.375000,39.878049,27.744186,12.195122,2016,1,Jan,Winter
2016-01-03,Mid Atlantic - Dominion,5247,1537,2936,774,947082.331,0.000000,0.000000,0.000000,7.850000,43.022222,25.600000,17.422222,2016,1,Jan,Winter
2016-01-04,Mid Atlantic - Dominion,4880,1537,2524,819,1130978.206,0.005106,0.105263,0.052632,10.600000,42.595745,23.085106,19.510638,2016,1,Jan,Winter
2016-01-05,Mid Atlantic - Dominion,6133,1542,1617,2974,1269061.592,0.010213,0.036842,0.052632,7.133333,29.212766,10.914894,18.297872,2016,1,Jan,Winter
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-29,Mid Atlantic - Dominion,7430,484,2672,4274,916089.631,0.073023,0.000000,0.000000,4.800000,56.860465,34.302326,22.558140,2019,12,Dec,Winter
2019-12-30,Mid Atlantic - Dominion,6608,484,3202,2922,943757.918,0.392558,0.000000,0.000000,10.383333,54.209302,41.255814,12.953488,2019,12,Dec,Winter
2019-12-31,Mid Atlantic - Dominion,6923,484,2722,3717,951694.034,0.106512,0.000000,0.000000,5.300000,54.581395,36.534884,18.046512,2019,12,Dec,Winter


### Visualize Weather Stations

In [ ]:
map_drop_cols = ['year','month','plot_date']
df_map = df_NOAA_Station_Trim.drop(columns=map_drop_cols).groupby('station').mean()
import plotly.express as px

fig = px.scatter_mapbox(df_map, lat="latitude", lon="longitude", hover_name=df_map.index,hover_data=df_map.columns,
                        color_discrete_sequence=["fuchsia"], zoom=3, height=700)
fig.update_layout(mapbox_style="open-street-map")
fig.show()